In [19]:
import pickle
from natsort import natsorted

basedir = '../../ImageNomer/data/anton/cohorts/test'
demoname = f'{basedir}/demographics.pkl'

with open(demoname, 'rb') as f:
    demo = pickle.load(f)
    
subs = natsorted(list(demo['age'].keys()))
print(len(subs))

830


In [34]:
import numpy as np

# y=0 NC y=1 SZ1 (130) y=2 SZ2 (69)

x = []
y = []

for sub in subs:
    for task in ['rest', 'nback', 'emoid']:
        if demo['race'][sub] == 'AA':
            y.append(1)
        elif demo['race'][sub] == 'EA':
            y.append(0)
        else:
            continue
#             print('Bad sex')
#             raise 'Bad'
        p = np.load(f'{basedir}/fc/{sub}_task-{task}_fc.npy')
        x.append(p)
    
x = np.stack(x)
y = np.array(y).astype('int')

print(x.shape)
print(y.shape)
print(y)

(2199, 34716)
(2199,)
[0 0 0 ... 1 1 1]


In [38]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=2, n_init='auto').fit(x[y != 0])
print(km.labels_)

ym = np.zeros(len(y))
ym[y != 0] = km.labels_ + 1

print(np.sum(ym == 1), np.sum(ym == 2))
print(ym)

[1 0 0 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 0
 0 0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1
 0 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1
 1 1 1 1 1 0 0 1 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 1
 1 1 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0
 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1
 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 1 0 1
 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1
 0 0 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 0 0
 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1
 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 1
 1 0 0 0 0 0 1 1 1 1 1 1 

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

idcs = np.arange(x.shape[0]) 
# idcs = np.where(ym != 1)[0]
xx = x[idcs]
yy = ym[idcs]
yy = yy != 0
yy = yy.astype('int')

accs = []

for _ in range(20):
    # Select balanced set
    a, b = np.sum(yy == 0), np.sum(yy == 1)
    num = 556 #min(a,b)
    g1x, g2x = xx[yy == 0], xx[yy == 1]
    g1idcs = np.random.permutation(g1x.shape[0])[0:num]
    g2idcs = np.random.permutation(g2x.shape[0])[0:num]
    xxx = np.concatenate([g1x[g1idcs], g2x[g2idcs]])
    yyy = np.concatenate([np.zeros(num), np.ones(num)])
    
    xtr, xt, ytr, yt = train_test_split(xx, yy, stratify=yy, train_size=0.8)

    print('Got to logistic')
    lr = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = lr.predict(xt)
    acc = np.mean(yhat == yt)
    print(acc)
    
    accs.append(acc)
    
print('---')
print(np.mean(accs))
print(np.std(accs))

Got to logistic
0.95
Got to logistic
0.95
Got to logistic
0.9318181818181818
Got to logistic
0.9363636363636364
Got to logistic
0.9318181818181818
Got to logistic
0.925
Got to logistic
0.9386363636363636
Got to logistic
0.9204545454545454
Got to logistic
0.9477272727272728
Got to logistic
0.9340909090909091
Got to logistic
0.9409090909090909
Got to logistic
0.925
Got to logistic
0.925
Got to logistic
0.9318181818181818
Got to logistic
0.9477272727272728
Got to logistic
0.9363636363636364
Got to logistic
0.9340909090909091
Got to logistic
0.9272727272727272
Got to logistic
0.95
Got to logistic
0.9545454545454546
---
0.936931818181818
0.009903329225672853


In [41]:
demo['AA_subtype'] = dict()

i = 0
for sub in subs:
    if demo['race'][sub] == 'AA':
        demo['AA_subtype'][sub] = 0
    for task in ['rest', 'nback', 'emoid']:
        if demo['race'][sub] not in ['EA', 'AA']:
            pass
        elif demo['race'][sub] == 'AA':
            if ym[i] == 1:
                demo['AA_subtype'][sub] -= 1
            elif ym[i] == 2:
                demo['AA_subtype'][sub] += 1
            else:
                print('Bad male')
                raise 'Bad'
            i += 1
        else:
            i += 1
#             print('Bad sex')
#             raise 'Bad'
        
print(len(demo['AA_subtype']))
print(demo['AA_subtype'])

326
{'600031697545': -1, '600054124128': -1, '600062084650': 1, '600085654611': 3, '600098941368': 1, '600110937794': -3, '600114922498': -3, '600129552715': -1, '600138617917': 1, '600185621034': 1, '600209995267': -1, '600210241146': -1, '600262185931': -3, '600263649795': 1, '600282088524': -1, '600294591042': 1, '600294620965': 3, '600325379088': 1, '600367706279': 3, '600377610905': 3, '600547831711': -3, '600557347302': -3, '600561317124': 1, '600587013053': 3, '600607081380': 3, '600647003585': 1, '600681250400': -3, '600682103788': 3, '600700496428': -1, '600710344290': -3, '600730262874': 1, '600748335231': 3, '600778703691': -3, '600790653076': -1, '600857437078': 1, '600883641613': -3, '600922891543': -1, '600928302123': -3, '600950857962': 3, '600994234394': 3, '601008078297': 1, '601031667414': -3, '601039605738': -1, '601064014005': -3, '601078429870': 3, '601091916731': 3, '601098855033': -1, '601102119981': 1, '601110916219': -1, '601121451354': 3, '601134535512': 3, '6

In [42]:
pickle.dump(demo, open(demoname, 'wb'))

print('Complete')

Complete
